# 이상치를 None(결측치)로 처리

In [1]:
import pandas as pd
import numpy as np
import os

## Grouping한 csv파일들 이름 load

In [2]:
csv_list=[]
new_address=[]

def get_csv_list(db_name,csv_list,new_address):
    list1=os.listdir("../RawData/{:s}/1hour".format(db_name))
    for i in range(len(list1)):
        csv_list.append("../RawData/{:s}/1hour/{:s}".format(db_name,list1[i]))
        new_address.append("../Grouped_Data/{:s}/{:s}/{:s}".format(db_name,'Missing_Value(All)',list1[i]))

    return csv_list,new_address

In [3]:
get_csv_list('air_indoor_도서관',csv_list,new_address)
get_csv_list('air_indoor_중학교',csv_list,new_address)
get_csv_list('air_indoor_체육시설',csv_list,new_address)
get_csv_list('air_indoor_초등학교',csv_list,new_address)

(['../RawData/air_indoor_도서관/1hour/ICW0W2000087.csv',
  '../RawData/air_indoor_도서관/1hour/ICW0W2000088.csv',
  '../RawData/air_indoor_도서관/1hour/ICW0W2000089.csv',
  '../RawData/air_indoor_도서관/1hour/ICW0W2000094.csv',
  '../RawData/air_indoor_도서관/1hour/ICW0W2000095.csv',
  '../RawData/air_indoor_도서관/1hour/ICW0W2000096.csv',
  '../RawData/air_indoor_도서관/1hour/ICW0W2000097.csv',
  '../RawData/air_indoor_도서관/1hour/ICW0W2000098.csv',
  '../RawData/air_indoor_도서관/1hour/ICW0W2000099.csv',
  '../RawData/air_indoor_도서관/1hour/ICW0W2000100.csv',
  '../RawData/air_indoor_도서관/1hour/ICW0W2000101.csv',
  '../RawData/air_indoor_도서관/1hour/ICW0W2000102.csv',
  '../RawData/air_indoor_도서관/1hour/ICW0W2000108.csv',
  '../RawData/air_indoor_도서관/1hour/ICW0W2000128.csv',
  '../RawData/air_indoor_도서관/1hour/ICW0W2000129.csv',
  '../RawData/air_indoor_중학교/1hour/ICW0W2000010.csv',
  '../RawData/air_indoor_중학교/1hour/ICW0W2000011.csv',
  '../RawData/air_indoor_중학교/1hour/ICW0W2000013.csv',
  '../RawData/air_indoor_중학교

## 이상치를 none(결측치)로 처리하기

In [4]:
def convert_to_None(address1,address2,whats,max_points):
    
    file=pd.read_csv(address1)
    
    for what, Max_Point in zip(whats, max_points):
        file[what] = pd.to_numeric(file[what], errors='coerce')
        file.loc[(file[what] > Max_Point) | (file[what] <= 0), what] = None
    
    file.to_csv(address2, index=False)
    
    return

In [5]:
whats = ['in_co2', 'in_pm10', 'in_voc']
max_points = [float(811.3333333333334), float(60.83527542372882), float(1665.0862499999998)]

for address1, address2 in zip(csv_list, new_address):
    convert_to_None(address1, address2, whats, max_points)


## 누락된 시간행 탐색

In [6]:
from dateutil.parser import parse

def Nan_time_check(address):
    file = pd.read_csv(address)

    start_time=file['time'][0]
    end_time=file['time'][len(file)-1]
    
    # 'time' 열을 datetime 형식으로 변환
    file['time'] = file['time'].apply(lambda x: parse(x))

    start_time = parse(start_time)
    end_time = parse(end_time)

    expected_rows = pd.DataFrame(pd.date_range(start=start_time, end=end_time, freq='1H'), columns=['time'])

    missing_rows = expected_rows[~expected_rows['time'].isin(file['time'])]


    return missing_rows

## 누락된 시간행을 채워넣기

In [7]:
def merge_and_sort_files(address):

    file_Nan = pd.read_csv(address)
    file_Nan = pd.DataFrame(file_Nan)
    
    missing_rows=Nan_time_check(address)
    missing_rows=pd.DataFrame(missing_rows)

    missing_rows['time']=None
    missing_rows['in_co2']=None
    missing_rows['in_humi']=None
    missing_rows['in_pm10']=None
    missing_rows['in_temp']=None
    missing_rows['in_voc']=None
    
    merged_file = pd.concat([file_Nan, missing_rows])
    merged_file['time'] = pd.to_datetime(merged_file['time'])
    merged_file.sort_values('time', inplace=True)
    merged_file.to_csv(address, index=False, header=True)
    
    return

In [8]:
for address in new_address:
    merge_and_sort_files(address)